In [79]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
from sklearn.model_selection import train_test_split

In [80]:
df = pd.read_json("final_df_sentiment.json")

In [81]:
rating_review = df[['reviewText','overall']]

In [82]:
rating_review.head()

,reviewText,overall
0,"Not much to write about here, but it does exac...",5
1,The product does exactly as it should and is q...,5
10,"Monster makes a wide array of cables, includin...",5
100,The listing for this item can be confusing if ...,5
1000,The tips of my fingers often get pretty dry an...,5


In [83]:
rating_review["opinion"] = rating_review["overall"].apply(lambda value: "good" if value >= 3 else "bad")

c:\users\balachandar\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [84]:
rating_review[rating_review.overall == 2].head(3)

,reviewText,overall,opinion
10027,Soooo bummed. I simply love the sound on these...,2,bad
10045,"I love Tascam gear, and have used it for nearl...",2,bad
10110,"The concept of this guitar stand is great, and...",2,bad


In [85]:
rating_review["reviewText"].head(3)

0     Not much to write about here, but it does exac...
1     The product does exactly as it should and is q...
10    Monster makes a wide array of cables, includin...
Name: reviewText, dtype: object

In [86]:
regular_expression = re.compile('[^a-z]+')
def process_text(review_text):
    if type(review_text) is str:
        review_text = review_text.lower()
        review_text = regular_expression.sub(' ', review_text).strip()
    return review_text

rating_review["processedReview"] = rating_review["reviewText"].apply(process_text)

c:\users\balachandar\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [87]:
rating_review.head(3)

,reviewText,overall,opinion,processedReview
0,"Not much to write about here, but it does exac...",5,good,not much to write about here but it does exact...
1,The product does exactly as it should and is q...,5,good,the product does exactly as it should and is q...
10,"Monster makes a wide array of cables, includin...",5,good,monster makes a wide array of cables including...


In [88]:
rating_review = rating_review.drop(['reviewText', 'overall'],1)
rating_review.head(5)

,opinion,processedReview
0,good,not much to write about here but it does exact...
1,good,the product does exactly as it should and is q...
10,good,monster makes a wide array of cables including...
100,good,the listing for this item can be confusing if ...
1000,good,the tips of my fingers often get pretty dry an...


In [89]:
train_data, test_data = train_test_split(rating_review, test_size=0.2)
(len(train_data), len(test_data))

(7774, 1944)

In [90]:
train_data.head(3)

,opinion,processedReview
2708,good,serves its purpose worked better when i added ...
6678,bad,i bought this for a practice amp for my bass g...
6542,good,works very well i have to of them each does a ...


In [94]:
#CountVectorizer to get bag of words model
count_vectorizer = CountVectorizer(min_df = 1, ngram_range = (1, 5))
X_train_bags = count_vectorizer.fit_transform(train_data["processedReview"])

#tfidf to bag of words
tfidf_transformer = TfidfTransformer()
X_train_data = tfidf_transformer.fit_transform(X_train_bags)

#getting ready test data
X_test_counts = count_vectorizer.transform(test_data["processedReview"])
X_test_data = tfidf_transformer.transform(X_test_counts)